In [1]:
# import packages
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns 
from sklearn.metrics import pairwise_distances
import statsmodels.api as sm

## Read the data

In [21]:
# read sp-level abundance table
abudB = pd.read_csv('../step2_KRAKEN_aggregation/results/Cross-Platform_set_results/BGI_final_abud_part2.csv')
abudI = pd.read_csv('../step2_KRAKEN_aggregation/results/Cross-Platform_set_results/Illumina_final_abud_part2.csv')

fs = pd.read_csv('input/good_samples_to_include.txt', header=None)
fs.columns = ['sample']
fs['bgi_sample'] = 'bgi_'+fs['sample']
fs['ill_sample'] = 'ill_'+fs['sample']
fs.tail(2)

,sample,bgi_sample,ill_sample
1349,VGP5J1,bgi_VGP5J1,ill_VGP5J1
1350,VPEB3X,bgi_VPEB3X,ill_VPEB3X


In [22]:
# eclude column wich present fs except 'name' column
abudB = abudB.set_index('name')
abudBf = abudB[abudB.columns.intersection(fs['bgi_sample'])]
abudI = abudI.set_index('name')
abudIf = abudI[abudI.columns.intersection(fs['ill_sample'])]

print('Final BGI sample set contains', abudBf.shape[1]-1, 'samples')
print('Final Illumina sample set contains', abudIf.shape[1]-1, 'samples')

Final BGI sample set contains 1350 samples
Final Illumina sample set contains 1350 samples


In [52]:
# skip first row
#abud = abudBf.iloc[1:]
#abud.set_index('rep_MAG_ID', inplace=True)
abud = abudIf.T
abud.reset_index(inplace=True)

In [53]:
# read the phenotypes
#path = '/Users/katerynapantiukh/Documents/1MyDisk/PhD/!MAIN_data/'
path = '/Users/ketpantuh/Documents/1MyDisk/PhD/!MAIN_data/' # home

import os
files = os.listdir(str(path)+'pheno/')
# remove hidden files
files = [file for file in files if not file.startswith('.')]
# remove first 5 characters from each file name
files = [file[6:] for file in files]
ph_list = [file[:-4] for file in files]

In [54]:
# drugs description
ph_names = pd.read_excel(str(path)+'pheno_description/phenotype_names.xlsx')
ph_names.head(2)

,code,name,type
0,B18,Chronic viral hepatitis,disease
1,B37,Candidiasis,disease


## Data preparation

In [55]:
abud.head(2)

name,index,CAG-390 sp000437015,CAG-390 sp900753295,CAG-390 sp003523225,CAG-448 sp003150135,CAG-448 sp000433415,CAG-448 sp944381085,UMGS731 sp900544985,HGM12713 sp900754175,Flemingibacterium sp900546315,...,DTU008 sp001421185,Methanofastidiosum sp023622545,Spyradosoma merdigallinarum,Bullifex sp944390045,Enterousia intestinigallinarum,SIG714 sp015063395,Eubacterium_T sp905214225,Pseudomonas citronellolis,Enterousia sp944393245,Enteromonas pullicola
0,ill_V97EBU,0.0373,0.0,0.0,0.02142,0.0,0.0,0.01238,0.00421,0.00331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ill_V1JCFD,0.0000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
# filter out sp with prevalence <1%
abud = abud.loc[:, abud.astype(bool).sum(axis=0) >= 21]

# perform CLR transformation for columns starting from H
abud.iloc[:, 1:] = abud.iloc[:, 1:].apply(lambda x: x/x.sum(), axis=1)
abud.to_csv('bgi_assembly_v1_SpLevel_RelAbund_CLR_1prev.csv')

# write stats
fspNumb = len(abud.columns)
perc = (len(abud)-fspNumb)/len(abud)*100

# Bonferoni corrected significance level
alpha = 0.05 / fspNumb

print(f"Initial bacteria species count is {len(abud)}.")
print(f"Final sp. number after filtering is {fspNumb}.")
print(f"{str(perc)[0:5]}% of bacteria species were filtered out.")
print(f"Significance level after Bonferoni correction is {alpha}.")

Initial bacteria species count is 1351.
Final sp. number after filtering is 864.
36.04% of bacteria species were filtered out.
Significance level after Bonferoni correction is 5.787037037037037e-05.


In [57]:
des = ph_names[ph_names['type'] == 'disease']['code'].to_list()
ph_des_sel = []
# create df with selected phenotypes and number of cases
sel_des_list = pd.DataFrame(columns=['code', 'phenotype', 'cases','controls'])

for ph_name in des:
    # read phenotype
    ph = pd.read_csv(str(path)+'/pheno/Pheno_'+ph_name+'.csv')
    if ph[ph_name].sum() > 100:
        # add value to list
        ph_des_sel.append(ph_name)
        # add value 
        new_row = {'code': ph_name, 'phenotype': ph_names.loc[ph_names['code'] == ph_name, 'name'].values[0],'cases': ph[ph_name].sum(),'controls': 1878 - ph[ph_name].sum()}

        # Append the new row using pd.concat
        sel_des_list = pd.concat([sel_des_list, pd.DataFrame([new_row])], ignore_index=True)

sel_des_list.to_excel('results/selected_phenotypes.xlsx')

perc2 = (len(des)-len(ph_des_sel))/len(des)*100

print(f"Initial number of disease phenotype is {len(des)}.")
print(f"Number of phenotype with more then 100 cases is {len(ph_des_sel)}.")
print(f"{str(perc2)[0:5]}% of phenotypes were filtered out.")

Initial number of disease phenotype is 66.
Number of phenotype with more then 100 cases is 33.
50.0% of phenotypes were filtered out.


In [58]:
# select raws present in des
desList = ph_names[ph_names['code'].isin(ph_des_sel)]
desList


,code,name,type
1,B37,Candidiasis,disease
5,D50,Iron deficiency anemia,disease
10,E55,Vitamin D deficiency,disease
12,E78,Disorders of lipoprotein metabolism and other ...,disease
15,F32,"Major depressive disorder, single episode",disease
16,F33,"Major depressive disorder, recurrent",disease
17,F41,Other anxiety disorders,disease
21,G43,Migraine,disease
22,H25,Age-related cataract,disease
23,H40,Glaucoma,disease


In [59]:
dr = ph_names[ph_names['type'] != 'disease']['code'].to_list()
ph_dr_sel = []

for ph_name in dr:
    # read phenotype
    ph = pd.read_csv(str(path)+'/pheno/Pheno_'+ph_name+'.csv')
    if ph[ph_name].sum() > 100:
        # add value to list
        ph_dr_sel.append(ph_name)

perc3 = (len(dr)-len(ph_dr_sel))/len(dr)*100

print(f"Initial number of drugs phenotype is {len(dr)}.")
print(f"Number of phenotype with more then 100 cases is {len(ph_dr_sel)}.")
print(f"{str(perc3)[0:5]}% of phenotypes were filtered out.")

Initial number of drugs phenotype is 132.
Number of phenotype with more then 100 cases is 1.
99.24% of phenotypes were filtered out.


## Start loop correlation analysis through the phenotypes

In [63]:
# one phenotype test
ph_name = 'D50'
# Define your covariates
covariates = ['Age_at_MBsample', 'BMI', 'gender']

pheno = pd.read_csv(str(path)+'pheno/Pheno_'+str(ph_name)+'.csv')
#pheno['index'] = 'bgi_'+pheno['vkood']
pheno['index'] = 'ill_'+pheno['vkood']
mrg = abud.merge(pheno, left_on='index', right_on='index', how='inner')

# perform CLR transformation for columns with bacteria species
tc = len(mrg.columns)-5

# fill NA values with mean
mrg['BMI'].fillna(mrg['BMI'].mean(), inplace=True)
mrg['gender'].fillna(mrg['gender'].mean(), inplace=True)
mrg['Age_at_MBsample'].fillna(mrg['Age_at_MBsample'].mean(), inplace=True)

# create a df with results for each phenotype
df = pd.DataFrame(columns=['bacteria', 'p-value', 'betta'])

n = len(mrg.columns)-5
bac = mrg.columns[1:n]
len(bac)

for b in bac:
    # Create a design matrix by adding the covariates to the model
    X = mrg[covariates + [ph_name]]
    X = sm.add_constant(X)

    y = mrg[b]
    model = sm.OLS(y, X).fit()
    if model.pvalues[ph_name] < alpha:
                print(b)
                print(model.pvalues[ph_name])  
                print(model.params[ph_name])
                print('-----------------')


## Run correlation analysis for all phenotypes

In [64]:
# phenotypes with more then 100 cases
nMore = []
# number of phenotypes with less then 100 cases
nLess = []

# Define your covariates
covariates = ['Age_at_MBsample', 'BMI', 'gender']
# create a new df for all correlation table
allCor = pd.DataFrame(columns=['pheno','bacteria', 'p-value', 'betta'])
# create a new df for aggregated results
res = pd.DataFrame(columns=['phenotype', 'ass_found', 'ass_bact','cases','controls'])

for ph_name in ph_des_sel:
    #reads phenotype data
    pheno = pd.read_csv(str(path)+'pheno/Pheno_'+str(ph_name)+'.csv')
    #pheno['index_pheno'] = 'bgi_'+pheno['vkood']
    pheno['index_pheno'] = 'ill_'+pheno['vkood']
    mrg = abud.merge(pheno, left_on='index', right_on='index_pheno', how='inner')

    # perform CLR transformation for columns with bacteria species
    tc = len(mrg.columns)-6
    #mrg.iloc[:, 1:tc] = mrg.iloc[:, 1:tc].apply(lambda x: x/x.sum(), axis=1)

    if len(mrg[mrg[ph_name] == 1]) <100:
        print("There are not enought samples for phenotype {}. Only {} cases found".format(ph_name, len(mrg[mrg[ph_name] == 1])))
        nLess.append(ph_name)
        print('---')
    else:
        print("WOW! It is enought samples for phenotype {}. We found {} cases!".format(ph_name, len(mrg[mrg[ph_name] == 1])))
        nMore.append(ph_name)
        print('...')

        # fill NA values with mean
        mrg['BMI'].fillna(mrg['BMI'].mean(), inplace=True)
        mrg['gender'].fillna(mrg['gender'].mean(), inplace=True)
        mrg['Age_at_MBsample'].fillna(mrg['Age_at_MBsample'].mean(), inplace=True)

        # create a df with results for each phenotype
        df = pd.DataFrame(columns=['bacteria', 'p-value', 'betta'])

        n = len(mrg.columns)-6
        bac = mrg.columns[1:n]
        len(bac)

        for b in bac:
            # Create a design matrix by adding the covariates to the model
            X = mrg[covariates + [ph_name]]
            X = sm.add_constant(X)

            y = mrg[b]

            # Fit the linear regression model
            model = sm.OLS(y, X).fit()
            # Fit the logistic regression model
            #model = sm.Logit(y, X).fit(method='bfgs', maxiter=1000) 
            #model = sm.Probit(y, X).fit(method='bfgs', maxiter=1000)

            if model.pvalues[ph_name] < alpha:
                print(b)
                #print(model.pvalues[ph_name])   

                # fill out separete table for phenotype
                rows_to_append = []
                new_row = {'bacteria': b, 'p-value': model.pvalues[ph_name], 'betta': model.params[ph_name]}
                rows_to_append.append(new_row)
                new_data = pd.DataFrame(rows_to_append)
                df = pd.concat([df, new_data], ignore_index=True)

                # fill out AllCor table
                rows_to_append = []
                new_row = {'pheno': ph_name, 'bacteria': b, 'p-value': model.pvalues[ph_name], 'betta': model.params[ph_name]}
                rows_to_append.append(new_row)
                new_data = pd.DataFrame(rows_to_append)
                allCor = pd.concat([allCor, new_data], ignore_index=True)

            df.sort_values(by=['p-value'], inplace=True)
            df.to_csv('results/Pval_'+str(ph_name)+'.csv', index=False)     
        
        if df.empty:
            print('There are no significant correlations')
            new_row = {'phenotype': ph_name, 'ass_found': 'no','ass_bact': '-','cases': len(mrg[mrg[ph_name] == 1]),'controls':len(mrg[mrg[ph_name] == 0])}
        else:
            bct = ', '.join(df['bacteria'].to_list())
            new_row = {'phenotype': ph_name, 'ass_found': 'yes','ass_bact': bct,'cases': len(mrg[mrg[ph_name] == 1]),'controls':len(mrg[mrg[ph_name] == 0])}

        rows_to_append = []
        rows_to_append.append(new_row)

        new_data = pd.DataFrame(rows_to_append)
        res = pd.concat([res, new_data], ignore_index=True)

        print('---')

# Write the results!
allCor = allCor.merge(ph_names, left_on='pheno', right_on='code', how='left')
allCor = allCor[['pheno','name','bacteria','p-value', 'betta']]
allCor.to_excel('results/allCorrelation_table_v2.xlsx', index=False) 

res = res.merge(ph_names, left_on='phenotype', right_on='code', how='left')
res = res[['code','name','ass_found','ass_bact','cases','controls']].sort_values(by='ass_found', ascending=False)
res.to_excel('results/aggrCorrelation_table_v2.xlsx', index=False) 


WOW! It is enought samples for phenotype B37. We found 284 cases!
...
There are no significant correlations
---
WOW! It is enought samples for phenotype D50. We found 180 cases!
...
There are no significant correlations
---
There are not enought samples for phenotype E55. Only 67 cases found
---
WOW! It is enought samples for phenotype E78. We found 269 cases!
...
There are no significant correlations
---
WOW! It is enought samples for phenotype F32. We found 262 cases!
...
There are no significant correlations
---
WOW! It is enought samples for phenotype F33. We found 118 cases!
...
Bacteroides faecis
Thauera sp009469595
---
WOW! It is enought samples for phenotype F41. We found 189 cases!
...
There are no significant correlations
---
There are not enought samples for phenotype G43. Only 91 cases found
---
WOW! It is enought samples for phenotype H25. We found 137 cases!
...
Escherichia coli
Escherichia sp002965065
Escherichia fergusonii
Escherichia sp004211955
Escherichia sp005843885

In [10]:
%%capture captured_output
k = len(res[res['ass_found'] == 'yes'])

# print current date
from datetime import datetime

current_date = datetime.now()
print(current_date, '\n')

print(f"Bacteria filter: sp. with prevalence <1%")
print(f"Initial bacteria species count is {len(abud)}.")
print(f"Final sp. number after filtering is {fspNumb}.")
print(f"{str(perc)[0:5]}% of bacteria species were filtered out.\n")

print(f"Data preparation: SLR transformation")
print(f"Significance level: Bonferoni correction")
print(f"Significance level after is {alpha}. \n")

print(f"Phenotype filter: filter out phenotypes with less then 100 cases")
print(f"It was analised 66 phenotypes")
print(f"Fot {len(nMore)} phenotypes with enought cases")
print(f"Among them {k} phenotypes had {len(allCor)} associations \n")

print(f"Phenotypes with associations:")
a = ', '.join(res[res['ass_found'] == 'yes']['name'].to_list())
print(a, '\n')

print(f"Phenotypes without associations:")
b = ', '.join(res[res['ass_found'] != 'yes']['name'].to_list())
print(b)

In [12]:
with open('results/log.csv', 'w') as f:
    f.write(captured_output.stdout)

## Run correlation analysis for all phenotypes
and store all pvalues

In [8]:
# phenotypes with more then 100 cases
nMore = []
# number of phenotypes with less then 100 cases
nLess = []

# Define your covariates
covariates = ['Age_at_MBsample', 'BMI', 'gender']
# create a new df for all correlation table
allCor = pd.DataFrame(columns=['pheno','bacteria', 'p-value', 'betta'])
# create a new df for aggregated results
res = pd.DataFrame(columns=['phenotype', 'ass_found', 'ass_bact','cases','controls'])

for ph_name in ph_des_sel:
    #reads phenotype data
    pheno = pd.read_csv(str(path)+'pheno/Pheno_'+str(ph_name)+'.csv')
    mrg = abud.merge(pheno, left_on='index', right_on='vkood', how='inner')

    # perform CLR transformation for columns with bacteria species
    tc = len(mrg.columns)-5
    #mrg.iloc[:, 1:tc] = mrg.iloc[:, 1:tc].apply(lambda x: x/x.sum(), axis=1)

    if len(mrg[mrg[ph_name] == 1]) <100:
        print("There are not enought samples for phenotype {}. Only {} cases found".format(ph_name, len(mrg[mrg[ph_name] == 1])))
        nLess.append(ph_name)
        print('---')
    else:
        print("WOW! It is enought samples for phenotype {}. We found {} cases!".format(ph_name, len(mrg[mrg[ph_name] == 1])))
        nMore.append(ph_name)
        print('...')

        # fill NA values with mean
        mrg['BMI'].fillna(mrg['BMI'].mean(), inplace=True)
        mrg['gender'].fillna(mrg['gender'].mean(), inplace=True)
        mrg['Age_at_MBsample'].fillna(mrg['Age_at_MBsample'].mean(), inplace=True)

        # create a df with results for each phenotype
        df = pd.DataFrame(columns=['bacteria', 'p-value', 'betta'])

        n = len(mrg.columns)-5
        bac = mrg.columns[1:n]
        len(bac)

        for b in bac:
            # Create a design matrix by adding the covariates to the model
            X = mrg[covariates + [ph_name]]
            X = sm.add_constant(X)

            y = mrg[b]

            # Fit the linear regression model
            model = sm.OLS(y, X).fit()
            # Fit the logistic regression model
            #model = sm.Logit(y, X).fit(method='bfgs', maxiter=1000) 
            #model = sm.Probit(y, X).fit(method='bfgs', maxiter=1000)  

            # fill out separete table for phenotype
            rows_to_append = []
            new_row = {'bacteria': b, 'p-value': model.pvalues[ph_name], 'betta': model.params[ph_name]}
            rows_to_append.append(new_row)
            new_data = pd.DataFrame(rows_to_append)
            df = pd.concat([df, new_data], ignore_index=True)

            # fill out AllCor table
            rows_to_append = []
            new_row = {'pheno': ph_name, 'bacteria': b, 'p-value': model.pvalues[ph_name], 'betta': model.params[ph_name]}
            rows_to_append.append(new_row)
            new_data = pd.DataFrame(rows_to_append)
            allCor = pd.concat([allCor, new_data], ignore_index=True)

            df.sort_values(by=['p-value'], inplace=True)
            df.to_csv('results/sm-Probit/Pval_'+str(ph_name)+'.csv', index=False)     
        
        if df.empty:
            print('There are no significant correlations')
            new_row = {'phenotype': ph_name, 'ass_found': 'no','ass_bact': '-','cases': len(mrg[mrg[ph_name] == 1]),'controls':len(mrg[mrg[ph_name] == 0])}
        else:
            bct = ', '.join(df['bacteria'].to_list())
            new_row = {'phenotype': ph_name, 'ass_found': 'yes','ass_bact': bct,'cases': len(mrg[mrg[ph_name] == 1]),'controls':len(mrg[mrg[ph_name] == 0])}

        rows_to_append = []
        rows_to_append.append(new_row)

        new_data = pd.DataFrame(rows_to_append)
        res = pd.concat([res, new_data], ignore_index=True)

print('---THE END---')



WOW! It is enought samples for phenotype B37. We found 407 cases!
...
---THE END---
WOW! It is enought samples for phenotype D50. We found 255 cases!
...
---THE END---
WOW! It is enought samples for phenotype E55. We found 102 cases!
...
---THE END---
WOW! It is enought samples for phenotype E78. We found 417 cases!
...
---THE END---
WOW! It is enought samples for phenotype F32. We found 392 cases!
...
---THE END---
WOW! It is enought samples for phenotype F33. We found 177 cases!
...
---THE END---
WOW! It is enought samples for phenotype F41. We found 287 cases!
...
---THE END---
WOW! It is enought samples for phenotype G43. We found 133 cases!
...
---THE END---
WOW! It is enought samples for phenotype H25. We found 209 cases!
...
---THE END---
WOW! It is enought samples for phenotype H40. We found 116 cases!
...
---THE END---
WOW! It is enought samples for phenotype I10. We found 457 cases!
...
---THE END---
WOW! It is enought samples for phenotype I11. We found 377 cases!
...
---THE